# Small RNN that predicts powers of 2

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

import numpy as np
import pandas as pd

# RNN Modell LSTM (Long Short Term Memory) Variantion von RNN
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


In [3]:
powers_of_two = [2**i for i in range(1, 15)]

data = []
# Erstellt Sequenzen mit Längen von 3, 4 und 5
for window_size in [3, 4, 5]:
    for i in range(len(powers_of_two) - window_size):
        seq = powers_of_two[i:i+window_size]
        target = powers_of_two[i+window_size]
        data.append((seq, target))

# Anwendung des Logarithmus
data_log = [(list(map(np.log, seq)), np.log(target)) for seq, target in data]
data_log = data_log * 3

pd.DataFrame(data).head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
0,"[2, 4, 8]","[4, 8, 16]","[8, 16, 32]","[16, 32, 64]","[32, 64, 128]","[64, 128, 256]","[128, 256, 512]","[256, 512, 1024]","[512, 1024, 2048]","[1024, 2048, 4096]"
1,16,32,64,128,256,512,1024,2048,4096,8192


In [4]:
model = SimpleRNN(input_size=1, hidden_size=64, output_size=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 1000
for epoch in tqdm(range(epochs)):
    total_loss = 0
    for sequence, target in data_log:
        sequence = torch.FloatTensor(sequence).view(1, -1, 1)
        target = torch.FloatTensor([target])
        optimizer.zero_grad()
        output = model(sequence)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}/{epochs}, Loss: {total_loss/len(data)}')


  0%|          | 0/1000 [00:00<?, ?it/s]/Users/ruben/anaconda3/envs/ba-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 3/1000 [00:00<01:09, 14.42it/s]

Epoch 0/1000, Loss: 53.36703467589687


100%|██████████| 1000/1000 [00:54<00:00, 18.38it/s]


In [6]:
def predict_next_power_of_2(sequence, model):
    sequence_log = list(map(np.log, sequence))
    tensor_input = torch.FloatTensor(sequence_log).view(1, -1, 1)
    predicted_log = model(tensor_input).item()
    return np.exp(predicted_log)

input_sequence_1 = [2, 4, 8]
input_sequence_2 = [2, 4, 8, 16]
input_sequence_3 = [2]
input_sequence_4 = [64, 128, 256, 512 ]

predicted_value_1 = predict_next_power_of_2(input_sequence_1, model)
predicted_value_2 = predict_next_power_of_2(input_sequence_2, model)
predicted_value_3 = predict_next_power_of_2(input_sequence_3, model)
predicted_value_4 = predict_next_power_of_2(input_sequence_4, model)


print(f'Predicted next power of 2 for {input_sequence_1}: {predicted_value_1}')
print(f'Predicted next power of 2 for {input_sequence_2}: {predicted_value_2}')
print(f'Predicted next power of 2 for {input_sequence_3}: {predicted_value_3}')
print(f'Predicted next power of 2 for {input_sequence_4}: {predicted_value_4}')

Predicted next power of 2 for [2, 4, 8]: 16.09371835180147
Predicted next power of 2 for [2, 4, 8, 16]: 31.483109891721593
Predicted next power of 2 for [2]: 7.33693101343991
Predicted next power of 2 for [64, 128, 256, 512]: 1003.9508464856866
